<a href="https://colab.research.google.com/github/ArkanDash/Advanced-RVC-Inference/blob/master/Advanced-RVC-NoUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced RVC Inference NoUI (Dev Mode)


[Discord](https://discord.gg/hvmsukmBHE) - [Github](https://github.com/ArkanDash/Advanced-RVC-Inference)



This is the development mode version of Advanced RVC Inference (No UI/CLI). Uses `git clone` for full development access.


In [ ]:
#@title Setup Environment
import os
import subprocess
from IPython.display import clear_output, display

#@markdown **Installation Method**
#@markdown - `local`: Use existing cloned repo (Advanced-RVC-Inference/)
#@markdown - `git+uv`: Install via uv from GitHub (faster)
#@markdown - `git+pip`: Install via pip from GitHub
install_method = "local"  #@param ["local", "git+uv", "git+pip"]

# Determine repo directory
repo_dir = "Advanced-RVC-Inference"
in_repo = False

match install_method:
    case "local":
        # Check if repo exists
        if os.path.exists(repo_dir):
            %cd $repo_dir
            !git pull -q origin master
            in_repo = True
            print(f"✅ Using existing repo: {repo_dir}/")
        else:
            # Fallback to git clone if local not found
            !git clone https://github.com/ArkanDash/Advanced-RVC-Inference.git
            %cd $repo_dir
            in_repo = True
            print(f"✅ Cloned repo to: {repo_dir}/")
    case "git+uv":
        # Install uv if not available
        !pip install uv -q
        # Install directly from git with uv
        !uv pip install git+https://github.com/ArkanDash/Advanced-RVC-Inference.git --system
        print("✅ Installed via git+uv")
    case "git+pip":
        # Install directly from git with pip
        !pip install git+https://github.com/ArkanDash/Advanced-RVC-Inference.git
        print("✅ Installed via git+pip")

# Install system dependencies for audio processing (always needed)
!apt-get -y install libportaudio2 ffmpeg -qq > /dev/null 2>&1

clear_output()
print("✅ Installation complete!")
if install_method == "local":
    print(f"Repository: {repo_dir}/")
print("Use the cells below for inference and training tasks.")

In [ ]:
#@title **Download Model** (CLI)
# Usage: python -m advanced_rvc_inference.core.downloads <link> <model_name>

import os
import subprocess
from IPython.display import clear_output, display
from ipywidgets import Button

#@markdown **Download model from URL (HuggingFace, Google Drive, Mega, Mediafire)**
model_link = ""  #@param {type:"string", placeholder:"https://huggingface.co/..."}
model_name = ""  #@param {type:"string", placeholder:"Model name"}

if model_link and model_name:
    cmd = f"python -m advanced_rvc_inference.core.downloads {model_link} {model_name}"
    !cd advanced_rvc_inference && $cmd
    clear_output()
    display(Button(description="✓ Model Downloaded!", button_style="success"))
else:
    print("⚠️ Please provide both model_link and model_name")
    print("Usage: python -m advanced_rvc_inference.core.downloads <link> <model_name>")
    print("\nOr upload manually to: advanced_rvc_inference/assets/weights/")


## Audio Separation (MDX-Net)


In [ ]:
#@title **Music Separation** (CLI)
# Usage: python -m advanced_rvc_inference.separate_music --input_path <path> --output_dirs <dir> --model_name <model>

import os
from IPython.display import clear_output, display
from ipywidgets import Button

#@markdown **Input path (URL or local path) and output directory**
input_path = ""  #@param {type:"string", placeholder:"/content/audio.wav or URL"}
output_dir = "separated"  #@param {type:"string"}
#@markdown **Separation model and options**
separation_model = "MDXNET_Main"  #@param ["Main_340", "Main_390", "Main_406", "Main_427", "Main_438", "Inst_full_292", "Inst_HQ_1", "Inst_HQ_2", "Inst_HQ_3", "Inst_HQ_4", "Inst_HQ_5", "Kim_Vocal_1", "Kim_Vocal_2", "Kim_Inst", "Inst_187_beta", "Inst_82_beta", "Inst_90_beta", "Voc_FT", "Crowd_HQ", "Inst_1", "Inst_2", "Inst_3", "MDXNET_1_9703", "MDXNET_2_9682", "MDXNET_3_9662", "Inst_Main", "MDXNET_Main", "MDXNET_9482", "HT-Normal", "HT-Tuned", "HD_MMI", "HT_6S"]
overlap = 0.25  #@param {type:"number", min:0.25, max:0.99, step:0.25}
segment_size = 256  #@param {type:"integer", min:32, max:2048, step:8}
export_format = "wav"  #@param ["wav", "mp3", "flac", "ogg", "opus", "m4a"]

if input_path:
    os.makedirs(output_dir, exist_ok=True)
    cmd = f"python -m advanced_rvc_inference.separate_music --input_path '{input_path}' --output_dirs '{output_dir}' --export_format {export_format}"
    !cd advanced_rvc_inference && $cmd
    clear_output()
    display(Button(description="✓ Separation Complete!", button_style="success"))
else:
    print("⚠️ Please provide input_path")
    print("\nUsage: python -m advanced_rvc_inference.separate_music --input_path <path> --output_dirs <dir>")


## Audio Conversion (RVC Inference)


In [ ]:
#@title **Audio Conversion** (CLI)
# Usage: python -m advanced_rvc_inference.convert --input_path <path> --pth_path <model> --output_path <out>

import os
from IPython.display import clear_output, display
from ipywidgets import Button

#@markdown **Model selection**
model_name = ""  #@param {type:"string", placeholder:"Model name (without .pth)"}
#@markdown **Pitch and input/output settings**
pitch_shift = 0  #@param {type:"integer", min:-20, max:20}
input_path = ""  #@param {type:"string", placeholder:"Input audio path"}
output_path = "output.wav"  #@param {type:"string"}
#@markdown **F0 extraction and processing options**
f0_method = "rmvpe"  #@param ["pm", "dio", "harvest", "yin", "pyin", "swipe", "crepe-tiny", "crepe-small", "crepe-medium", "crepe-large", "crepe-full", "fcpe", "rmvpe", "rmvpe-legacy"]
hop_length = 64  #@param {type:"integer", min:64, max:512}
index_rate = 0.5  #@param {type:"number", min:0, max:1}
protect = 0.33  #@param {type:"number", min:0, max:1}
#@markdown **Output format**
export_format = "wav"  #@param ["wav", "mp3", "flac", "ogg", "opus", "m4a"]

if model_name and input_path:
    model_path = f"advanced_rvc_inference/assets/weights/{model_name}.pth"
    
    # Find index file
    index_path = ""
    for root, _, files in os.walk("advanced_rvc_inference/assets/logs"):
        for f in files:
            if f.endswith(".index") and "trained" not in f and model_name in f:
                index_path = os.path.join(root, f)
                break
    
    cmd = f"python -m advanced_rvc_inference.convert --input_path '{input_path}' --pth_path '{model_path}' --output_path '{output_path}' --pitch {pitch_shift} --f0_method {f0_method} --hop_length {hop_length} --index_rate {index_rate} --protect {protect} --export_format {export_format}"
    !cd advanced_rvc_inference && $cmd
    clear_output()
    display(Button(description="✓ Conversion Complete!", button_style="success"))
else:
    print("⚠️ Please provide model_name and input_path")
    print("\nUsage: python -m advanced_rvc_inference.convert --input_path <path> --pth_path <model.pth> --output_path <out>")


## Text-to-Speech (TTS) Conversion


In [ ]:
#@title **Text-to-Speech Conversion** (CLI)

import os
from IPython.display import clear_output, display
from ipywidgets import Button

#@markdown **Text input**
text_input = "Hello, this is a test of the text-to-speech conversion."  #@param {type:"string"}
#@markdown **Voice selection (Microsoft Azure Neural Voices)**
voice = "en-US-AvaNeural"  #@param ["en-US-AvaNeural", "en-US-AndrewNeural", "en-US-EmmaNeural", "en-US-BrianNeural", "en-US-GuyNeural", "en-US-JennyNeural", "en-US-ChristopherNeural"]
speech_rate = 0  #@param {type:"integer", min:-100, max:100}
#@markdown **Model and pitch settings**
model_name = ""  #@param {type:"string", placeholder:"Model name (without .pth)"}
pitch_shift = 0  #@param {type:"integer", min:-20, max:20}
output_format = "wav"  #@param ["wav", "mp3", "flac"]

tts_output = f"tts_raw.{output_format}"
convert_output = f"tts_converted.{output_format}"

if model_name and text_input:
    model_path = f"advanced_rvc_inference/assets/weights/{model_name}.pth"
    
    # Find index file
    index_path = ""
    for root, _, files in os.walk("advanced_rvc_inference/assets/logs"):
        for f in files:
            if f.endswith(".index") and "trained" not in f and model_name in f:
                index_path = os.path.join(root, f)
                break
    
    # Generate TTS
    cmd1 = f"python -m advanced_rvc_inference.tts --text '{text_input}' --voice '{voice}' --rate {speech_rate} --output '{tts_output}'"
    !cd advanced_rvc_inference && $cmd1
    
    # Convert with RVC model
    cmd2 = f"python -m advanced_rvc_inference.convert --input_path '{tts_output}' --pth_path '{model_path}' --output_path '{convert_output}' --pitch {pitch_shift} --f0_method rmvpe"
    if index_path:
        cmd2 += f" --index_path '{index_path}' --index_rate 0.5"
    !cd advanced_rvc_inference && $cmd2
    
    clear_output()
    print(f"✓ TTS Conversion Complete! Output: {convert_output}")
    display(Button(description="✓ Done!", button_style="success"))
else:
    print("⚠️ Please provide model_name and text_input")


## Dataset Creation


In [ ]:
#@title **Create Training Dataset** (CLI)
# Downloads audio from YouTube or other sources and preprocesses it

import os
from IPython.display import clear_output, display
from ipywidgets import Button

#@markdown **Audio source (YouTube URL or audio files)**
audio_source = ""  #@param {type:"string", placeholder:"YouTube URL or comma-separated URLs"}
output_folder = "dataset"  #@param {type:"string"}
#@markdown **Sample rate and processing options**
sample_rate = 48000  #@param ["40000", "48000", "50000"]
#@markdown **Music separation for vocals**
separate_audio = true  #@param {type:"boolean"}

if audio_source:
    os.makedirs(output_folder, exist_ok=True)
    cmd = f"python -m advanced_rvc_inference.create_dataset --input_data '{audio_source}' --output_dirs '{output_folder}' --sample_rate {sample_rate}"
    if separate_audio:
        cmd += " --separate"
    !cd advanced_rvc_inference && $cmd
    clear_output()
    display(Button(description="✓ Dataset Created!", button_style="success"))
else:
    print("⚠️ Please provide audio_source (YouTube URL)")
    print("\nUsage: python -m advanced_rvc_inference.create_dataset --input_data <url> --output_dirs <folder>")


## Feature Extraction


In [ ]:
#@title **Extract Features** (CLI)
# Extracts pitch (F0) and embedding features from training audio

import os
from IPython.display import clear_output, display
from ipywidgets import Button

#@markdown **Model information**
model_name = ""  #@param {type:"string", placeholder:"Model name"}
rvc_version = "v2"  #@param ["v1", "v2"]
#@markdown **Processing options**
f0_method = "rmvpe"  #@param ["pm", "dio", "harvest", "yin", "pyin", "swipe", "crepe-tiny", "crepe-small", "crepe-medium", "crepe-large", "crepe-full", "fcpe", "rmvpe", "rmvpe-legacy"]
sample_rate = 48000  #@param {type:"integer"}

if model_name:
    cmd = f"python -m advanced_rvc_inference.extract --model_name '{model_name}' --rvc_version {rvc_version} --f0_method {f0_method} --sample_rate {sample_rate}"
    !cd advanced_rvc_inference && $cmd
    clear_output()
    display(Button(description="✓ Features Extracted!", button_style="success"))
else:
    print("⚠️ Please provide model_name")
    print("\nUsage: python -m advanced_rvc_inference.extract --model_name <name> --sample_rate <sr>")


## Preprocessing


In [ ]:
#@title **Preprocess Training Data** (CLI)
# Slices audio and resamples to 16kHz for training

import os
from IPython.display import clear_output, display
from ipywidgets import Button

#@markdown **Model and dataset settings**
model_name = ""  #@param {type:"string", placeholder:"Model name"}
dataset_path = "dataset"  #@param {type:"string", placeholder:"Path to audio files"}
sample_rate = 48000  #@param {type:"integer"}
#@markdown **Processing options**
cpu_cores = 2  #@param {type:"integer", min:1, max:16}
cut_preprocess = "Automatic"  #@param ["Automatic", "Simple", "Skip"]

if model_name:
    cmd = f"python -m advanced_rvc_inference.preprocess --model_name '{model_name}' --dataset_path '{dataset_path}' --sample_rate {sample_rate} --cpu_cores {cpu_cores} --cut_preprocess {cut_preprocess}"
    !cd advanced_rvc_inference && $cmd
    clear_output()
    display(Button(description="✓ Preprocessing Complete!", button_style="success"))
else:
    print("⚠️ Please provide model_name")
    print("\nUsage: python -m advanced_rvc_inference.preprocess --model_name <name> --dataset_path <path>")


## Training


In [ ]:
#@title **Train Model** (CLI)

import os
from IPython.display import clear_output, display
from ipywidgets import Button

#@markdown **Model configuration**
model_name = ""  #@param {type:"string", placeholder:"Model name"}
rvc_version = "v2"  #@param ["v1", "v2"]
model_author = ""  #@param {type:"string", placeholder:"Your name (optional)"}
#@markdown **Training schedule**
total_epoch = 300  #@param {type:"integer"}
save_every_epoch = 50  #@param {type:"integer"}
save_only_latest = true  #@param {type:"boolean"}
#@markdown **Batch and hardware settings**
batch_size = 8  #@param {type:"integer", min:1}
gpu = "0"  #@param {type:"string", placeholder:"0 or 0-1-2 for multiple GPUs"}
#@markdown **Advanced options**
pitch_guidance = true  #@param {type:"boolean"}
cache_data_in_gpu = false  #@param {type:"boolean"}

if model_name:
    cmd = f"python -m advanced_rvc_inference.training --train --model_name '{model_name}' --rvc_version {rvc_version} --total_epoch {total_epoch} --save_every_epoch {save_every_epoch} --save_only_latest {save_only_latest} --batch_size {batch_size} --gpu {gpu} --pitch_guidance {pitch_guidance} --cache_data_in_gpu {cache_data_in_gpu}"
    
    if model_author:
        cmd += f" --model_author '{model_author}'"
    
    !cd advanced_rvc_inference && $cmd
    clear_output()
    display(Button(description="✓ Training Complete!", button_style="success"))
else:
    print("⚠️ Please provide model_name")
    print("\nUsage: python -m advanced_rvc_inference.training --train --model_name <name> --total_epoch <epochs>")


## Create Index


In [ ]:
#@title **Create Index File** (CLI)
# Creates the .index file for faster inference

import os
from IPython.display import clear_output, display
from ipywidgets import Button

#@markdown **Model information**
model_name = ""  #@param {type:"string", placeholder:"Model name"}
rvc_version = "v2"  #@param ["v1", "v2"]
algorithm = "Auto"  #@param ["Auto", "Faiss", "KMeans"]

if model_name:
    cmd = f"python -m advanced_rvc_inference.create_index --model_name '{model_name}' --rvc_version {rvc_version} --index_algorithm {algorithm}"
    !cd advanced_rvc_inference && $cmd
    clear_output()
    display(Button(description="✓ Index Created!", button_style="success"))
else:
    print("⚠️ Please provide model_name")
    print("\nUsage: python -m advanced_rvc_inference.create_index --model_name <name>")


## Audio Effects


In [ ]:
#@title **Add Audio Effects** (CLI)

import os
from IPython.display import clear_output, display
from ipywidgets import Button

#@markdown **Input and output paths**
input_path = ""  #@param {type:"string", placeholder:"Input audio path"}
output_path = "output_effects.wav"  #@param {type:"string"}
#@markdown **Effect options**
reverb_room_size = 0.3  #@param {type:"number", min:0, max:1}
reverb_damping = 0.4  #@param {type:"number", min:0, max:1}
reverb_wet_gain = 0  #@param {type:"number", min:-20, max:20}
reverb_wet_only = false  #@param {type:"boolean"}

if input_path:
    cmd = f"python -m advanced_rvc_inference.audio_effects --input_path '{input_path}' --output_path '{output_path}' --reverb_room_size {reverb_room_size} --reverb_damping {reverb_damping} --reverb_wet_gain {reverb_wet_gain} --reverb_wet_only {reverb_wet_only}"
    !cd advanced_rvc_inference && $cmd
    clear_output()
    display(Button(description="✓ Effects Applied!", button_style="success"))
else:
    print("⚠️ Please provide input_path")
    print("\nUsage: python -m advanced_rvc_inference.audio_effects --input_path <path> --output_path <out>")


## Reference Set Creation


In [ ]:
#@title **Create Reference Set** (CLI)
# Creates a reference set for consistent voice cloning results

import os
from IPython.display import clear_output, display
from ipywidgets import Button

#@markdown **Audio file and settings**
audio_path = ""  #@param {type:"string", placeholder:"Reference audio path"}
reference_name = "my_reference"  #@param {type:"string"}
#@markdown **Model settings**
model_name = ""  #@param {type:"string", placeholder:"Model name (for settings)"}
rvc_version = "v2"  #@param ["v1", "v2"]

if audio_path and model_name:
    cmd = f"python -m advanced_rvc_inference.create_reference --audio_path '{audio_path}' --reference_name '{reference_name}' --model_name '{model_name}' --version {rvc_version}"
    !cd advanced_rvc_inference && $cmd
    clear_output()
    display(Button(description="✓ Reference Created!", button_style="success"))
else:
    print("⚠️ Please provide audio_path and model_name")
    print("\nUsage: python -m advanced_rvc_inference.create_reference --audio_path <path> --reference_name <name> --model_name <model>")


## Play Audio Output


In [ ]:
#@title **Play Audio**

from IPython.display import Audio, display

audio_path = ""  #@param {type:"string", placeholder:"Path to audio file"}

if audio_path:
    display(Audio(audio_path))
else:
    print("Please provide audio_path")


## Help & Usage


In [ ]:
#@title **Show Help**

from IPython.display import clear_output

#@markdown **Select command to get help**
help_command = "--help"  #@param ["--help", "--help_convert", "--help_create_dataset", "--help_create_index", "--help_extract", "--help_preprocess", "--help_separate_music", "--help_train", "--help_create_reference"]

cmd = f"python -m advanced_rvc_inference {help_command}"
!cd advanced_rvc_inference && $cmd
